In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import struct

Using TensorFlow backend.


In [2]:
train_data_fileName="train-images.idx3-ubyte"
train_label_fileName="train-labels.idx1-ubyte"
test_data_fileName="t10k-images.idx3-ubyte"
test_label_fileName="t10k-labels.idx1-ubyte"

In [3]:
def readData(fileName):
    with open(fileName,'rb') as obj:
        data=obj.read(16)
        magic,N,n_row,n_col=struct.unpack('!4i',data)
        images=np.zeros((N,n_row,n_col))
        for i in range(N):
            data=obj.read(n_row*n_col)
            images[i]=np.array(struct.unpack('!'+str(n_row*n_col)+'B',data)).reshape((n_row,n_col))
    return images
def readLabels(fileName):
    with open(fileName,'rb') as obj:
        data=obj.read(8)
        magic,N=struct.unpack('!2i',data)
        data=obj.read(N)
        labels=np.array(struct.unpack('!'+str(N)+'B',data))
    return labels

In [4]:
train_data=readData(train_data_fileName)
N,n_row,n_col=train_data.shape
train_data=train_data.reshape((N,n_row,n_col,1))
train_labels=np.eye(10)[readLabels(train_label_fileName)]

test_data=readData(test_data_fileName)
N,n_row,n_col=test_data.shape
test_data=test_data.reshape((N,n_row,n_col,1))
test_labels=np.eye(10)[readLabels(test_label_fileName)]


In [5]:

train_data=np.concatenate([train_data,255-train_data],axis=0)
train_labels=np.concatenate([train_labels,train_labels],axis=0)

test_data=np.concatenate([test_data,255-test_data],axis=0)
test_labels=np.concatenate([test_labels,test_labels],axis=0)

In [6]:
def Model(input_size=(28,28,1)):
    initial_method="random_uniform"
    X_Input=keras.layers.Input(input_size)
    X=keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation='relu',name='C1',padding='same',kernel_initializer=initial_method)(X_Input)
    X=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),name='S2')(X)
    X=keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation='relu',name='C3',kernel_initializer=initial_method)(X)
    X=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),name='S4')(X)
    X=keras.layers.Flatten(name='flatten')(X)
    X=keras.layers.Dense(120,activation='relu',name='C5',kernel_initializer=initial_method)(X)
    X=keras.layers.Dense(84,activation='relu',name='C6',kernel_initializer=initial_method)(X)
    X=keras.layers.Dense(10,activation='softmax',name='Output',kernel_initializer=initial_method)(X)
    model=keras.models.Model(inputs=X_Input,outputs=X,name='myModel')
    return model
   

In [7]:
model=Model()
model.compile(metrics=['accuracy'],optimizer='adam',loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
optimizer_name='adam_uniform_2'
checkpoint_best = keras.callbacks.ModelCheckpoint("mnist_"+optimizer_name+".h5", monitor='val_acc',verbose=1,save_best_only=False)

hist=model.fit(train_data,train_labels,
         batch_size=128,
         epochs=10,
         shuffle=True,
         validation_data=(test_data,test_labels),
         callbacks=[checkpoint_best])


Instructions for updating:
Use tf.cast instead.
Train on 120000 samples, validate on 20000 samples
Epoch 1/10
120000/120000 [==============================] - 9s 77us/step - loss: 0.1762 - acc: 0.9436 - val_loss: 0.0679 - val_acc: 0.9771

Epoch 00001: saving model to mnist_adam_uniform_2.h5
Epoch 2/10
120000/120000 [==============================] - 7s 60us/step - loss: 0.0553 - acc: 0.9827 - val_loss: 0.0425 - val_acc: 0.9857

Epoch 00002: saving model to mnist_adam_uniform_2.h5
Epoch 3/10
120000/120000 [==============================] - 7s 60us/step - loss: 0.0430 - acc: 0.9868 - val_loss: 0.0387 - val_acc: 0.9877

Epoch 00003: saving model to mnist_adam_uniform_2.h5
Epoch 4/10
120000/120000 [==============================] - 7s 60us/step - loss: 0.0364 - acc: 0.9888 - val_loss: 0.0559 - val_acc: 0.9838

Epoch 00004: saving model to mnist_adam_uniform_2.h5
Epoch 5/10
120000/120000 [==============================] - 7s 60us/step - loss: 0.0319 - acc: 0.9901 - val_loss: 0.0449 - val_ac

In [70]:
import h5py
fileName=optimizer_name+"_hist.h5"
with h5py.File(fileName) as obj:
    obj.create_dataset('val_loss',data=hist.history['val_loss'])
    obj.create_dataset('val_acc',data=hist.history['val_acc'])
    obj.create_dataset('loss',data=hist.history['loss'])
    obj.create_dataset('acc',data=hist.history['acc'])

In [73]:
fileName="sgd_hist.h5"
with h5py.File(fileName,'r') as obj:
    print(obj['val_loss'][:])

[0.10300788 0.09113237 0.06137469 0.05830885 0.06718337 0.04471642
 0.05229546 0.04317136 0.04272618 0.04586303]


In [87]:
negative_test_data=255-test_data
negative_predict=model.predict(negative_test_data)
print(np.argmax(negative_predict,axis=1))
print(np.argmax(test_labels,axis=1))
right=np.argmax(negative_predict,axis=1)==np.argmax(test_labels,axis=1)
acc=len(right)/len(test_labels)
print(acc)

[7 6 7 ... 7 7 6]
[7 2 1 ... 4 5 6]
1.0
